In [1]:
# Import necessary libraries
import numpy as np
import pandas as pd
from datetime import datetime
import pandas as pd
import numpy as np
import seaborn as sns
import os
import random
import matplotlib
import matplotlib.pyplot as plt
from scipy import sparse
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import mean_squared_error
import warnings
warnings.filterwarnings('ignore')

In [2]:
import json
pd.set_option("display.max_columns", None)

## Loading datasets

In [3]:
#json data file path
business_json_path = 'yelp_academic_dataset_business.json'
review_json_path = 'yelp_academic_dataset_review.json'
user_json_path = 'yelp_academic_dataset_user.json'
tip_json_path = 'yelp_academic_dataset_tip.json'
checkin_json_path = 'yelp_academic_dataset_checkin.json'

In [4]:
#function to read the json data
def load_rows(filepath, nrows = None):
    with open(filepath,encoding='utf-8') as json_file:
        count = 0
        objs = []
        line = json_file.readline()
        while (nrows is None or count < nrows) and line:
            count += 1
            obj = json.loads(line)
            objs.append(obj)
            line = json_file.readline()
        return pd.DataFrame(objs)

In [55]:
def preprocess_business_data(df_business):
    # Getting only the open businesses
    df_business = df_business[df_business['is_open']==1]
    
    # getting only restaurent data
    mask = df_business.categories.str.contains('Restaurant', na=False) | df_business.categories.str.contains('Food', na=False)
    df_business= df_business[mask]
    
    # Setting the unique id for each businesses
    businessid_to_idx = {b_id : idx for idx, b_id in enumerate(df_business.business_id.unique())}
    df_business.business_id = df_business.business_id.map(lambda x: businessid_to_idx[x])
    
    attr = [col for col in df_business.attributes.explode().unique() if col is not None]
    lst_of_attr_dict = []
    for attr_dict in df_business.attributes:
        if not attr_dict:
            lst_of_attr_dict.append({})
            continue

        if 'BusinessParking' in attr_dict:
            if type(attr_dict['BusinessParking']) == str:
                attr_dict['BusinessParking'] = ('True' in attr_dict['BusinessParking'])

        lst_of_attr_dict.append(attr_dict)

    attr_df = pd.DataFrame(lst_of_attr_dict, columns=attr)
    # Handling missing data
    for col in attr_df:
        attr_df[col] = attr_df[col].fillna(False).astype(bool)

    df_business = pd.concat([df_business.reset_index().drop('index', axis=1), attr_df], axis=1)
    df_business.drop(['attributes'], axis=1, inplace=True)
    
    # getting the business hours
    lst_of_time = []
    for time_dict in df_business.hours:
        if not time_dict:
            lst_of_time.append({})
            continue
        lst_of_time.append(time_dict)
    time_df = pd.DataFrame(lst_of_time)
    
    df_business = pd.concat([df_business, time_df], axis=1).drop('hours', axis=1)

    return df_business

In [57]:
df_business=load_rows(business_json_path)

In [58]:
df_business_cleaned = preprocess_business_data(df_business)
df_business_cleaned.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,categories,RestaurantsTableService,WiFi,BikeParking,BusinessParking,BusinessAcceptsCreditCards,RestaurantsReservations,WheelchairAccessible,Caters,OutdoorSeating,RestaurantsGoodForGroups,HappyHour,BusinessAcceptsBitcoin,RestaurantsPriceRange2,Ambience,HasTV,Alcohol,GoodForMeal,DogsAllowed,RestaurantsTakeOut,NoiseLevel,RestaurantsAttire,RestaurantsDelivery,GoodForKids,ByAppointmentOnly,Music,GoodForDancing,BestNights,BYOB,CoatCheck,Smoking,DriveThru,Corkage,BYOBCorkage,RestaurantsCounterService,AcceptsInsurance,DietaryRestrictions,AgesAllowed,Open24Hours,HairSpecializesIn,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday
0,0,Oskar Blues Taproom,921 Pearl St,Boulder,CO,80302,40.017544,-105.283348,4.0,86,1,"Gastropubs, Food, Beer Gardens, Restaurants, B...",True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,11:0-23:0,11:0-23:0,11:0-23:0,11:0-23:0,11:0-23:0,11:0-23:0,11:0-23:0
1,1,Flying Elephants at PDX,7000 NE Airport Way,Portland,OR,97218,45.588906,-122.593331,4.0,126,1,"Salad, Soup, Sandwiches, Delis, Restaurants, C...",False,True,True,True,True,True,False,True,True,True,False,False,True,True,True,True,True,False,True,True,True,True,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,5:0-18:0,5:0-17:0,5:0-18:0,5:0-18:0,5:0-18:0,5:0-18:0,5:0-18:0
2,2,Bob Likes Thai Food,3755 Main St,Vancouver,BC,V5V,49.251342,-123.101333,3.5,169,1,"Restaurants, Thai",False,True,True,True,False,True,False,True,True,True,False,False,True,True,True,True,True,True,True,True,True,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,17:0-21:0,17:0-21:0,17:0-21:0,17:0-21:0,17:0-21:0,17:0-21:0,17:0-21:0
3,3,Boxwood Biscuit,740 S High St,Columbus,OH,43206,39.947007,-82.997471,4.5,11,1,"Breakfast & Brunch, Restaurants",False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,NaN,NaN,NaN,NaN,NaN,8:0-14:0,8:0-14:0
4,4,Chautauqua General Store,100 Clematis Dr,Boulder,CO,80302,39.998449,-105.281006,3.5,5,1,"Food, Shopping, Convenience Stores, Souvenir S...",False,False,True,True,True,False,True,True,False,False,False,True,False,False,False,False,False,True,True,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,8:0-20:0,8:0-20:0,8:0-20:0,8:0-20:0,8:0-20:0,8:0-20:0,8:0-20:0


In [59]:
# Saving the business dataframe to pickle
df_business_cleaned.to_pickle('business_df.pkl')

In [62]:
def preprocess_user_data(df_user):
    # changing the user ids
    userid_idx = {userid : idx for idx, userid in enumerate(df_user.user_id.unique())}
    df_user.user_id = df_user.user_id.map(lambda x: userid_idx[x])
    
    # changing the friends ids
    df_user.friends = df_user.friends.map(lambda x : str([userid_idx[user] for user in x.split(',') if user in userid_idx]))
    
    # Changing to datetime format
    df_user.yelping_since = pd.to_datetime(df_user.yelping_since)
    
    # Handling missing data
    df_user.elite = df_user.elite.replace('', np.nan)
    
    return df_user
    

In [6]:
df_user=load_rows(user_json_path)

,user_id,name,review_count,yelping_since,useful,funny,cool,elite,friends,fans,average_stars,compliment_hot,compliment_more,compliment_profile,compliment_cute,compliment_list,compliment_note,compliment_plain,compliment_cool,compliment_funny,compliment_writer,compliment_photos
0,q_QQ5kBBwlCcbL1s4NVK3g,Jane,1220,2005-03-14 20:26:35,15038,10030,11291,"2006,2007,2008,2009,2010,2011,2012,2013,2014","xBDpTUbai0DXrvxCe3X16Q, 7GPNBO496aecrjJfW6UWtg...",1357,3.85,1710,163,190,361,147,1212,5691,2541,2541,815,323
1,dIIKEfOgo0KqUfGQvGikPg,Gabi,2136,2007-08-10 19:01:51,21272,10289,18046,"2007,2008,2009,2010,2011,2012,2013,2014,2015,2...","XPzYf9_mwG2eXYP2BAGSTA, 2LooM5dcIk2o01nftYdPIg...",1025,4.09,1632,87,94,232,96,1187,3293,2205,2205,472,294
2,D6ErcUnFALnCQN4b1W_TlA,Jason,119,2007-02-07 15:47:53,188,128,130,"2010,2011","GfB6sC4NJQvSI2ewbQrDNA, jhZtzZNNZJOU2YSZ6jPlXQ...",16,3.76,22,1,3,0,0,5,20,31,31,3,1
3,JnPIjvC0cmooNDfsa9BmXg,Kat,987,2009-02-09 16:14:29,7234,4722,4035,"2009,2010,2011,2012,2013,2014","HQZPQhKMwRAyS6BCselVWQ, kP2U1s_sjQfHO9grxiyDTA...",420,3.77,1180,129,93,219,90,1120,4510,1566,1566,391,326
4,37Hc8hr3cw0iHLoPzLK6Ow,Christine,495,2008-03-03 04:57:05,1577,727,1124,"2009,2010,2011","-Q88pZUcrfN0BLBDp-bkAQ, etPn4Pv1Gc4cRZjRgB_BOw...",47,3.72,248,19,32,16,15,77,131,310,310,98,44


In [63]:
df_user_cleaned = preprocess_user_data(df_user)
df_user_cleaned.head()

,user_id,name,review_count,yelping_since,useful,funny,cool,elite,friends,fans,average_stars,compliment_hot,compliment_more,compliment_profile,compliment_cute,compliment_list,compliment_note,compliment_plain,compliment_cool,compliment_funny,compliment_writer,compliment_photos
0,0,Jane,1220,2005-03-14 20:26:35,15038,10030,11291,"2006,2007,2008,2009,2010,2011,2012,2013,2014",[463996],1357,3.85,1710,163,190,361,147,1212,5691,2541,2541,815,323
1,1,Gabi,2136,2007-08-10 19:01:51,21272,10289,18046,"2007,2008,2009,2010,2011,2012,2013,2014,2015,2...",[],1025,4.09,1632,87,94,232,96,1187,3293,2205,2205,472,294
2,2,Jason,119,2007-02-07 15:47:53,188,128,130,"2010,2011",[1094856],16,3.76,22,1,3,0,0,5,20,31,31,3,1
3,3,Kat,987,2009-02-09 16:14:29,7234,4722,4035,"2009,2010,2011,2012,2013,2014",[],420,3.77,1180,129,93,219,90,1120,4510,1566,1566,391,326
4,4,Christine,495,2008-03-03 04:57:05,1577,727,1124,"2009,2010,2011",[],47,3.72,248,19,32,16,15,77,131,310,310,98,44


In [64]:
df_user_cleaned.to_pickle('user_df.pkl')

In [7]:
df_tip=load_rows(tip_json_path)
df_tip.head()

,user_id,business_id,text,date,compliment_count
0,WCjg0jdHXMlwbqS9tZUx8Q,ENwBByjpoa5Gg7tKgxqwLg,Carne asada chips...,2011-07-22 19:07:35,0
1,42-Z02y9bABShAGZhuSzrQ,jKO4Og6ucdX2-YCTKQVYjg,Best happy hour from 3pm to 6pm! $1 off martin...,2014-09-10 07:33:29,0
2,5u7E3LYp_3eB8dLuUBazXQ,9Bto7mky640ocgezVKSfVg,"Nice people, skilled staff, clean location - b...",2013-12-13 23:23:41,0
3,wDWoMG5N9oI4DJ-p7z8EBg,XWFjKtRGZ9khRGtGg2ZvaA,"1/2-price bowling & the ""Very"" Old Fashion are...",2017-07-11 23:07:16,0
4,JmuFlorjjRshHTKzTwNtgg,mkrx0VhSMU3p3uhyJGCoWA,"Solid gold's. Great sauna. Great staff, too. E...",2016-11-30 08:46:36,0


In [8]:
df_checkin=load_rows(checkin_json_path)
df_checkin.head()

,business_id,date
0,--0r8K_AQ4FZfLsX3ZYRDA,2017-09-03 17:13:59
1,--0zrn43LEaB4jUWTQH_Bg,"2010-10-08 22:21:20, 2010-11-01 21:29:14, 2010..."
2,--164t1nclzzmca7eDiJMw,"2010-02-26 02:06:53, 2010-02-27 08:00:09, 2010..."
3,--2aF9NhXnNVpDV0KS3xBQ,"2014-11-03 16:35:35, 2015-01-30 18:16:03, 2015..."
4,--2mEJ63SC_8_08_jGgVIg,"2010-12-15 17:10:46, 2013-12-28 00:27:54, 2015..."


In [9]:
reviews = []
r_dtypes = {"stars": np.float16, 
            "useful": np.int32, 
            "funny": np.int32,
            "cool": np.int32,
           }
with open("yelp_academic_dataset_review.json", "r",encoding='utf-8') as f:
    reader = pd.read_json(f, orient="records", lines=True, 
                          dtype=r_dtypes, chunksize=1000)
        
    for chunk in reader:
        #reduced_chunk = chunk.drop(columns=['review_id', 'user_id'])
        reviews.append(chunk)
    
reviews = pd.concat(reviews, ignore_index=True)

In [10]:
reviews.head()

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,lWC-xP3rd6obsecCYsGZRg,ak0TdVmGKo4pwqdJSTLwWw,buF9druCkbuXLX526sGELQ,4.0,3,1,1,Apparently Prides Osteria had a rough summer a...,2014-10-11 03:34:02
1,8bFej1QE5LXp4O05qjGqXA,YoVfDbnISlW0f7abNQACIg,RA4V8pr014UyUbDvI-LW2A,4.0,1,0,0,This store is pretty good. Not as great as Wal...,2015-07-03 20:38:25
2,NDhkzczKjLshODbqDoNLSg,eC5evKn1TWDyHCyQAwguUw,_sS2LBIGNT5NQb6PD1Vtjw,5.0,0,0,0,I called WVM on the recommendation of a couple...,2013-05-28 20:38:06
3,T5fAqjjFooT4V0OeZyuk1w,SFQ1jcnGguO0LYWnbbftAA,0AzLzHfOJgL7ROwhdww2ew,2.0,1,1,1,I've stayed at many Marriott and Renaissance M...,2010-01-08 02:29:15
4,sjm_uUcQVxab_EeLCqsYLg,0kA0PAJ8QFMeveQWHFqz2A,8zehGz9jnxPqXtOc7KaJxA,4.0,0,0,0,The food is always great here. The service fro...,2011-07-28 18:05:01


In [68]:
print(f'Cleaned business df shape: {df_business_cleaned.shape}')
print(f'Cleaned users df shape: {df_user_cleaned.shape}')
print(f'Tips df shape: {df_tip.shape}')
print(f'Checkin df shape: {df_checkin.shape}')
print(f'Reviews df shape: {reviews.shape}')

Cleaned business df shape: (41768, 58)
Cleaned users df shape: (2189457, 22)
Tips df shape: (1162119, 5)
Checkin df shape: (138876, 2)
Reviews df shape: (4725630, 9)


In [12]:
reviews.shape

(8635403, 9)

## Cleaning the Data

In [69]:
df_business_cleaned.shape

(41768, 58)

In [ ]:
userid_to_idx = {u_id : idx for idx, u_id in enumerate(reviews.user_id.unique())}
businessid_to_idx = {b_id : idx for idx, b_id in enumerate(reviews.business_id.unique())}

reviews.user_id = reviews.user_id.map(lambda x: userid_to_idx[x])
reviews.business_id = reviews.business_id.map(lambda x: businessid_to_idx[x])

In [70]:
#checking null values
df_business_cleaned.isnull().sum()

business_id                      0
name                             0
address                          0
city                             0
state                            0
postal_code                      0
latitude                         0
longitude                        0
stars                            0
review_count                     0
is_open                          0
categories                       0
RestaurantsTableService          0
WiFi                             0
BikeParking                      0
BusinessParking                  0
BusinessAcceptsCreditCards       0
RestaurantsReservations          0
WheelchairAccessible             0
Caters                           0
OutdoorSeating                   0
RestaurantsGoodForGroups         0
HappyHour                        0
BusinessAcceptsBitcoin           0
RestaurantsPriceRange2           0
Ambience                         0
HasTV                            0
Alcohol                          0
GoodForMeal         

In [25]:
reviews.shape

(8635403, 9)

Check for reviews of only the businesses that are still open

In [74]:
reviews = reviews[reviews.business_id.isin(df_business_cleaned.business_id.values)]
reviews.shape

(4725630, 9)

In [75]:
#checking for null values
reviews.isnull().sum()

review_id      0
user_id        0
business_id    0
stars          0
useful         0
funny          0
cool           0
text           0
date           0
dtype: int64

In [37]:
#checking data types and shape
reviews.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4725630 entries, 1 to 8635402
Data columns (total 9 columns):
 #   Column       Dtype         
---  ------       -----         
 0   review_id    object        
 1   user_id      object        
 2   business_id  object        
 3   stars        float16       
 4   useful       int32         
 5   funny        int32         
 6   cool         int32         
 7   text         object        
 8   date         datetime64[ns]
dtypes: datetime64[ns](1), float16(1), int32(3), object(4)
memory usage: 279.4+ MB


In [38]:
reviews.head()

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
1,8bFej1QE5LXp4O05qjGqXA,YoVfDbnISlW0f7abNQACIg,RA4V8pr014UyUbDvI-LW2A,4.0,1,0,0,This store is pretty good. Not as great as Wal...,2015-07-03 20:38:25
3,T5fAqjjFooT4V0OeZyuk1w,SFQ1jcnGguO0LYWnbbftAA,0AzLzHfOJgL7ROwhdww2ew,2.0,1,1,1,I've stayed at many Marriott and Renaissance M...,2010-01-08 02:29:15
5,J4a2TuhDasjn2k3wWtHZnQ,RNm_RWkcd02Li2mKPRe7Eg,xGXzsc-hzam-VArK6eTvtw,1.0,2,0,0,"This place used to be a cool, chill place. Now...",2018-01-21 04:41:03
7,9vqwvFCBG3FBiHGmOHMmiA,XGkAG92TQ3MQUKGX9sLUhw,DbXHNl890xSXNiyRczLWAg,5.0,0,0,0,Probably one of the better breakfast sandwiche...,2017-12-02 18:16:13
12,Gi5LSRmTXoL9Bp4jNGPjLw,hn0ZbitvmlHnF--KJGJ6_A,TA1KUSCu8GkWP9w0rmElxw,4.0,0,0,0,I have been here twice and have had really goo...,2011-10-27 14:32:57


In [80]:
reviews.to_pickle('reviews_df.pkl')

In [41]:
import time

start = time.time()
reviews_df = pd.read_pickle('reviews_df.pkl')
end = time.time()
print(f'Total time: {end-start}')

Total time: 40.516430377960205


In [79]:
reviews.head()

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
1,8bFej1QE5LXp4O05qjGqXA,0,0,4.0,1,0,0,This store is pretty good. Not as great as Wal...,2015-07-03 20:38:25
3,T5fAqjjFooT4V0OeZyuk1w,1,1,2.0,1,1,1,I've stayed at many Marriott and Renaissance M...,2010-01-08 02:29:15
5,J4a2TuhDasjn2k3wWtHZnQ,2,2,1.0,2,0,0,"This place used to be a cool, chill place. Now...",2018-01-21 04:41:03
7,9vqwvFCBG3FBiHGmOHMmiA,3,3,5.0,0,0,0,Probably one of the better breakfast sandwiche...,2017-12-02 18:16:13
12,Gi5LSRmTXoL9Bp4jNGPjLw,4,4,4.0,0,0,0,I have been here twice and have had really goo...,2011-10-27 14:32:57


In [81]:
final_df = reviews[['user_id', 'business_id', 'stars']]
final_df.head()

,user_id,business_id,stars
1,0,0,4.0
3,1,1,2.0
5,2,2,1.0
7,3,3,5.0
12,4,4,4.0


In [82]:
final_df.to_pickle('final_df.pkl')